Импортируем нужные библиотеки

In [1]:
import torch
import mlflow
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.loggers import MLFlowLogger

Импортируем необходимые классы

In [2]:
import config
from dataset import DataModule
from model import RankingModel

Задаем сиды, чтобы можно было воспроизводить результаты

In [3]:
torch.random.manual_seed(config.RANDOM_SEED)
pl.seed_everything(config.RANDOM_SEED)

Seed set to 42


42

In [4]:
loggers = []

Подключаем `TensorBoardLogger` логирование

In [5]:
if config.TENSORBOARD_LOGGER:
    tensorboard_logger = TensorBoardLogger(save_dir="tb_logs", name="ranking_model")
    loggers.append(tensorboard_logger)

Подключаем `MLFlow`

In [6]:
import os
from dotenv import load_dotenv

In [7]:
load_dotenv()
os.environ["MLFLOW_S3_ENDPOINT_URL"] = config.MLFLOW_S3_ENDPOINT_URL

In [8]:
if config.MLFLOW_LOGGER:
    mlf_logger = MLFlowLogger(
        experiment_name=config.EXPERIMENT_NAME,
        tracking_uri=config.TRACKING_URL,
        log_model=config.LOG_MODEL,
    )
    loggers.append(mlf_logger)

Создаем необходимые инстанты: данные, модель, учитель.

In [9]:
dm = DataModule(
    config.TRAIN_DATA_PATH,
    config.TEST_DATA_PATH,
    config.BATCH_SIZE,
    config.NUM_WORKERS,
    config.TRAIN_VAL_RATIO,
)
model = RankingModel(config.INPUT_SIZE, config.LEARNING_RATE)
trainer = pl.Trainer(
    accelerator=config.ACCELERATOR,
    devices=config.DEVICES,
    min_epochs=config.MIN_EPOCHS,
    max_epochs=config.MAX_EPOCHS,
    logger=loggers,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Обучаем модель, и тестируем и валидируем.

In [10]:
trainer.fit(model, dm)
trainer.validate(model, dm)
trainer.test(model, dm)


  | Name    | Type    | Params
------------------------------------
0 | fc1     | Linear  | 3.8 K 
1 | fc2     | Linear  | 32.9 K
2 | fc3     | Linear  | 129   
3 | loss_fn | BCELoss | 0     
------------------------------------
36.9 K    Trainable params
0         Non-trainable params
36.9 K    Total params
0.147     Total estimated model params size (MB)


/Users/evlko/Documents/GitHub/VK-MLE/.venv/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 12: 100%|██████████| 9/9 [00:00<00:00, 62.76it/s, v_num=71c4] 

`Trainer.fit` stopped: `max_epochs=13` reached.


Validation DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 36.73it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss            0.6278309226036072
        val_ndcg            0.8412059545516968
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing DataLoader 0: 100%|██████████| 24/24 [00:00<00:00, 189.90it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.5728029012680054
        test_ndcg      

[{'test_loss': 0.5728029012680054, 'test_ndcg': 0.5769645571708679}]

Сохраняем модель, чтобы использовать её в проде

In [11]:
script = model.to_torchscript()
torch.jit.save(script, config.MODEL_PATH)

Проверим размер модели

In [12]:
print("model size: {:.3f}MB".format(model.get_model_size()))

model size: 0.141MB


В целом модель занимает мало места, можно не делать квантизацию или пост-пруннинг.

In [13]:
if config.SAVE_MODEL:
    with mlflow.start_run():
        mlflow.pytorch.log_model(model, "models", registered_model_name=config.MODEL_NAME)

Successfully registered model 'VK_MODEL'.
2024/03/11 12:09:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VK_MODEL, version 1
Created version '1' of model 'VK_MODEL'.
